https://www.youtube.com/watch?v=pX7vlnG6eC8&t=7s
https://github.com/bluefeversoft/flappy_learning_pygame 

In [1]:
import numpy as np
import pygame
import scipy.spatial
import random

pygame 2.1.3 (SDL 2.0.22, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Constants
DISPLAY_w = 960
DISPLAY_h = 540
FPS = 30

DATA_FONT_SIZE = 18
DATA_FONT_COLOR = (40, 40, 40)
BG_FILENAME = "BG.png"

PIPE_FILENAME = "Pipe.png"
PIPE_SPEED = 70/1000
PIPE_DOWN = 1
PIPE_MOVING = 0
PIPE_UPPER = 1
PIPE_LOWER = 0

PIPE_ADD_GAP = 160
PIPE_MIN = 80
PIPE_MAX = 500
PIPE_START_X = DISPLAY_w
PIPE_GAP_SIZE = 160
PIPE_FIRST = 400

BIRD_FILENAME = "Robin.png"
BIRD_START_SPEED = -0.32
BIRD_START_X = 200
BIRD_START_Y = 200
BIRD_ALIVE = 1
BIRD_DEAD = 0
GRAVITY = 0.001

GENERATION_SIZE = 60

NNET_INPUTS = 2
NNET_HIDDEN = 5
NNET_OUTPUTS = 1

JUMP_CHANCE = 0.5

MAX_Y_DIFF = DISPLAY_h - PIPE_MIN - PIPE_GAP_SIZE/2
MIN_Y_DIFF = PIPE_GAP_SIZE/2 - PIPE_MAX
Y_SHIFT = abs(MIN_Y_DIFF)
NORMALIZER = abs(MIN_Y_DIFF) + MAX_Y_DIFF



In [3]:
class Nnet():

    def __init__(self, num_input, num_hidden, num_output):
        self.num_input = num_input
        self.num_hidden = num_hidden
        self.num_output = num_output
        self.weight_input_hidden = np.random.uniform(-0.5, 0.5, size=(self.num_hidden, self.num_input))
        self.weight_hidden_output = np.random.uniform(-0.5, 0.5, size=(self.num_output, self.num_hidden))
        self.activation_function = lambda x: scipy.special.expit(x)

    def get_outputs(self, inputs_list):
        inputs = np.array(inputs_list, ndmin=2).T
        hidden_inputs = np.dot(self.weight_input_hidden, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_outputs = np.dot(self.weight_hidden_output, hidden_outputs)
        return final_outputs

    def get_max_value(self, inputs_list):
        outputs = self.get_outputs(inputs_list)
        return np.max(outputs)

In [4]:
class Pipe():

    def __init__(self, gameDisplay, x, y, pipe_type):
        self.gameDisplay = gameDisplay
        self.state = PIPE_MOVING
        self.pipe_type = pipe_type
        self.img = pygame.image.load(PIPE_FILENAME)
        self.rect = self.img.get_rect()
        if pipe_type == PIPE_UPPER:
            y = y - self.rect.height
        self.set_position(x, y)
        print("Pipe moving")
    
    def set_position(self, x, y):
        self.rect.x = x
        self.rect.y = y
    
    def move_position(self, dx, dy):
        self.rect.centerx += dx
        self.rect.centery += dy

    def draw(self):
        self.gameDisplay.blit(self.img, self.rect)

    def check_status(self):
        if self.rect.right < 0:
            self.state = PIPE_DOWN
            print("Pipe is down")
    
    def update(self, dt):
        if self.state == PIPE_MOVING:
            self.move_position(-PIPE_SPEED*dt, 0)
            self.draw()
            self.check_status()
            

In [5]:
class PipsCollection():

    def __init__(self, gameDisplay):
        self.gameDisplay = gameDisplay
        self.pipes = []
        

    def add_new_pipe_pair(self, x):
        top_y = random.randint(PIPE_MIN, PIPE_MAX - PIPE_GAP_SIZE)
        bottom_y = top_y + PIPE_GAP_SIZE

        pi1 = Pipe(self.gameDisplay, x, top_y, PIPE_UPPER)
        pi2 = Pipe(self.gameDisplay, x, bottom_y, PIPE_LOWER)

        self.pipes.append(pi1)
        self.pipes.append(pi2)

    def create_new_set(self):
        self.pipes = []
        placed = PIPE_FIRST

        while placed < DISPLAY_w:
            self.add_new_pipe_pair(placed)
            placed += PIPE_ADD_GAP
    
    def update(self, dt):

        rightmost = 0

        for p in self.pipes:
            p.update(dt)
            if p.pipe_type == PIPE_UPPER:
                if p.rect.right > rightmost:
                    rightmost = p.rect.right
        
        if rightmost < (DISPLAY_w - PIPE_ADD_GAP):
            self.add_new_pipe_pair(DISPLAY_w)
    
        self.pipes = [p for p in self.pipes if p.state == PIPE_MOVING]

In [6]:
class Bird():

    def __init__(self, gameDisplay):
        self.gameDisplay = gameDisplay
        self.state = BIRD_ALIVE
        self.img = pygame.image.load(BIRD_FILENAME)
        self.rect = self.img.get_rect()
        self.speed = 0
        self.time_lived = 0
        self.nnet = Nnet(NNET_INPUTS, NNET_HIDDEN, NNET_OUTPUTS)
        self.set_position(BIRD_START_X, BIRD_START_Y)
        
    def set_position(self, x, y):
        self.rect.centerx = x
        self.rect.centery = y

    def move(self, dt):

        distance = 0
        new_speed = 0

        distance = (self.speed*dt) + (0.5*GRAVITY*dt*dt)
        new_speed = self.speed + GRAVITY*dt
        
        self.rect.centery += distance
        self.speed = new_speed

        if self.rect.top < 0:
            self.rect.top = 0
            self.speed = 0
        
    
    def jump(self, pipes):
        inputs = self.get_inputs(pipes)
        val1 = self.nnet.get_max_value(inputs)
        if val1 > JUMP_CHANCE:
            self.speed = BIRD_START_SPEED

    def draw(self):
        self.gameDisplay.blit(self.img, self.rect)

    def check_status(self, pipes):
        if self.rect.bottom > DISPLAY_h:
            self.state = BIRD_DEAD
        else:
            self.check_hits(pipes)
    
    def check_hits(self, pipes):
        for p in pipes:
            if p.rect.colliderect(self.rect):
                self.state = BIRD_DEAD
                break
    
    def update(self, dt, pipes):
        if self.state == BIRD_ALIVE:
            self.time_lived += dt
            self.move(dt)
            self.jump(pipes)
            self.draw()
            self.check_status(pipes)

    def get_inputs(self, pipes):

        closet = DISPLAY_h * 2
        bottom_y = 0
        for p in pipes:
            if p.pipe_type == PIPE_UPPER:
                if p.rect.right > self.rect.left and p.rect.left < closet:
                    closet = p.rect.left
                    bottom_y = p.rect.bottom
        
        horizontal_distance = closet - self.rect.left
        vertical_distance = (self.rect.centery) - (bottom_y + PIPE_GAP_SIZE/2)

        inputs = [
            ((horizontal_distance / DISPLAY_w) * 0.99)+ 0.01,
            (((vertical_distance + Y_SHIFT) / NORMALIZER) * 0.99) + 0.01
            ]

        return inputs

            

        

In [7]:
class BirdCollection():

    def __init__(self, gameDisplay):
        self.gameDisplay = gameDisplay
        self.birds = []
        self.create_new_generation()

    def create_new_generation(self):
        self.birds = []
        for i in range(0, GENERATION_SIZE):
            self.birds.append(Bird(self.gameDisplay))
    
    def update(self, dt, pipes):
        num_alive = 0
        for b in self.birds:
            b.update(dt, pipes)
            if b.state == BIRD_ALIVE:
                num_alive += 1
        
        return num_alive

In [8]:
def update_label(data, title, font, x, y, gameDisplay):
    label = font.render('{} {}'.format(title, data), 1, DATA_FONT_COLOR)
    gameDisplay.blit(label, (x, y))
    return y

In [9]:
def update_data_labels(gameDisplay, dt, game_time, num_interations, num_alive, font):
    y_pos = 10
    gap = 20
    x_pos = 10
    y_pos = update_label(round(1000/dt, 2), 'FPS', font, x_pos, y_pos + gap, gameDisplay)
    y_pos = update_label(round(game_time/1000, 2), 'Game time', font, x_pos, y_pos + gap, gameDisplay)
    y_pos = update_label(num_interations, 'Iterations', font, x_pos, y_pos + gap, gameDisplay)
    y_pos = update_label(num_alive, 'Alive', font, x_pos, y_pos + gap, gameDisplay)
    

In [10]:
def run_game():

    pygame.init()
    gameDisplay = pygame.display.set_mode((DISPLAY_w, DISPLAY_h))
    pygame.display.set_caption('Fågel')

    running = True
    bgImg = pygame.image.load(BG_FILENAME)
    pipes = PipsCollection(gameDisplay)
    pipes.create_new_set()
    birds = BirdCollection(gameDisplay)

    label_font = pygame.font.SysFont("Arial", DATA_FONT_SIZE)

    clock = pygame.time.Clock()
    dt = 0
    game_time = 0
    num_interations = 1

    

    while running:

        dt = clock.tick(FPS)
        game_time += dt

        gameDisplay.blit(bgImg, (0, 0))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    running = False

        
        pipes.update(dt)
        num_alive = birds.update(dt, pipes.pipes)
        
        if num_alive == 0:
            pipes.create_new_set()
            game_time = 0
            birds.create_new_generation()
            num_interations += 1

        
        update_data_labels(gameDisplay, dt, game_time, num_interations, num_alive, label_font)
        
        pygame.display.update()


In [11]:
if __name__ == "__main__":
    run_game()

Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe moving
Pipe

: 